In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_supra',
    'f_in_out_transaction_spr',
    proc_date,
    'temp_f_in_out_transaction_spr'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_supra.f_in_out_transaction_spr(
	wh_code STRING,
    wh_site_id STRING,
	client_code STRING,
	object_code STRING,
	object_type STRING,
	tran_date TIMESTAMP,
	job_code STRING,
  	job_type STRING,
	sku STRING,
	qty DOUBLE,
	uom STRING,
	barcode STRING,
	base_qty DOUBLE,
	base_uom STRING,
	expired_date DATE,
	manufacture_date DATE,
	best_before_date DATE,
	receive_date DATE,
	condition_type STRING,
	calday DATE,
	proc_date TIMESTAMP,
	tran_id STRING,
	file_creation_ts TIMESTAMP
	)
	TBLPROPERTIES 
	(
		DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE = TRUE,
		DELTA.AUTOOPTIMIZE.AUTOCOMPACT = TRUE
	)
""")

In [0]:
spark.sql(f"""DELETE FROM {catalog_name}.udp_wcm_silver_supra.f_in_out_transaction_spr 
WHERE tran_id IN (
  SELECT tran_id 
  FROM temp_f_in_out_transaction_spr
)""")

In [0]:
spark.sql(f"""
    INSERT INTO {catalog_name}.udp_wcm_silver_supra.f_in_out_transaction_spr (
        wh_code,
        wh_site_id,
        client_code,
        object_code,
        object_type,
        tran_date,
        job_code,
        job_type,
        sku,
        qty,
        uom,
        barcode,
        base_qty,
        base_uom,
        expired_date,
        manufacture_date,
        best_before_date,
        receive_date,
        condition_type,
        calday,
        proc_date,
        tran_id,
        file_creation_ts
    )
    SELECT DISTINCT
        wh_code,
        wh_site_id,
        client_code,
        object_code,
        object_type,
        TRY_CAST(tran_date AS DATE) AS tran_date,
        job_code,
        job_type,
        sku,
        qty,
        uom,
        barcode,
        base_qty,
        base_uom,
        TRY_CAST(expired_date AS DATE) AS expired_date,
        TRY_CAST(manufacture_date AS DATE) AS manufacture_date,
        TRY_CAST(best_before_date AS DATE) AS best_before_date,
        TRY_CAST(receive_date AS DATE) AS receive_date,
        condition_type,
        calday,
        TRY_CAST(proc_date AS DATE) AS proc_date,
        tran_id,
        file_creation_ts
    FROM temp_f_in_out_transaction_spr
    WHERE tran_id IS NOT NULL
""")